## CONFIG

In [5]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd
from bs4 import BeautifulSoup



from collections import Counter

from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [6]:
codice_galattico = "../Hackapizza Dataset/Codice Galattico/Codice Galattico.pdf"
manuale_cucina = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"

domande = "../Hackapizza Dataset/domande.csv"
piatti = "../Hackapizza Dataset/Misc/dish_mapping.json"
pianeti = "../Hackapizza Dataset/Misc/Distanze.csv"

blog_1 = "../Hackapizza Dataset/Blogpost/blog_etere_del_gusto.html"
blog_2 = "../Hackapizza Dataset/Blogpost/blog_sapore_del_dune.html"


load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [7]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [8]:
def estrai_testo_pdf(percorso_pdf):
    with open(percorso_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        testo = ""
        for pagina in reader.pages:
            testo += pagina.extract_text()
    return testo

def estrai_testo_html(percorso_html):
    """Estrae il testo da un file HTML."""
    with open(percorso_html, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        return soup.get_text(separator=" ", strip=True)

## METODI

In [14]:
def estrai_valorizzazioni(testo, categoria, llm, driver):
    # Genera un prompt basato sulle entità filtrate     
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti e Tecniche.
    
    Il testo che segue potrebbe contenere informazioni pertinenti a questa categoria: {categoria}
    In output voglio tutte le possibile istanze di questa categoria, come entità da caricare su neo4j.
    Per ogni istanza recupera:
        - NOME dell'istanza
        - eventuali PARAMETRI associati all'istanza
    Attento che i nomi delle chiavi devono essere identiche a quelle fornite.
    
    Restituisci SOLO il JSON valido e completo, senza formattazione markdown o decoratori.
    
    Testo:
    {testo}
    """
    
    # Chiamata al modello LLM per ottenere la valorizzazione delle entità
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('```json'):
        content = content[7:].strip()
    elif content.startswith('json'):
        content = content[4:].strip()
    
    # Tentativo di riparare il JSON
    try:
        valorizzazioni = json.loads(content)
    except json.JSONDecodeError as e:
        error_message = str(e)
        print(f"Errore nel parsing JSON: {error_message}")
        
        # Strategia 1: Estrai le parti valide e tenta di ricostruire
        if "Unterminated string" in error_message:
            try:
                # Metodo più sicuro: raccogliamo i dati che possiamo
                result = []
                import re
                
                # Cerca tutti i blocchi con NOME
                pattern = r'\{\s*"NOME":\s*"([^"]+)"'
                matches = re.finditer(pattern, content)
                
                for match in matches:
                    try:
                        # Estrai il frammento dal NOME fino alla fine del blocco o fino all'errore
                        start_pos = match.start()
                        # Tenta di trovare la chiusura del blocco
                        next_match_pos = content.find('"NOME":', match.end())
                        if next_match_pos == -1:
                            # È l'ultimo elemento
                            fragment = content[start_pos:]
                        else:
                            # C'è un altro elemento dopo
                            prev_pos = content.rfind('},', start_pos, next_match_pos)
                            if prev_pos != -1:
                                fragment = content[start_pos:prev_pos+2]
                            else:
                                fragment = content[start_pos:next_match_pos-1]
                        
                        # Verifica se il frammento è valido o riparalo
                        try:
                            # Prova a bilanciare le parentesi
                            balanced_fragment = fragment
                            if balanced_fragment.count('{') > balanced_fragment.count('}'):
                                balanced_fragment += '}' * (balanced_fragment.count('{') - balanced_fragment.count('}'))
                            
                            # Chiudi eventuali stringhe non terminate
                            if '"' in balanced_fragment:
                                open_quotes = balanced_fragment.count('"') % 2
                                if open_quotes:
                                    balanced_fragment += '"'
                            
                            # Prova a farne un JSON valido
                            obj = json.loads(balanced_fragment)
                            result.append(obj)
                        except:
                            # Se fallisce, prova con un approccio più semplice
                            nome_match = re.search(r'"NOME":\s*"([^"]+)"', fragment)
                            if nome_match:
                                result.append({"NOME": nome_match.group(1), "PARAMETRI": {}})
                    except Exception as inner_e:
                        print(f"Errore nell'estrazione di un elemento: {inner_e}")
                
                valorizzazioni = result
                print(f"Riparazione parziale completata. Elementi recuperati: {len(valorizzazioni)}")
            except Exception as repair_e:
                print(f"Impossibile riparare il JSON: {repair_e}")
                valorizzazioni = []
        else:
            # Tentativo di riparazione per altri tipi di errori
            fixed_content = content
            if content.count("[") > content.count("]"):
                fixed_content = fixed_content + "]" * (content.count("[") - content.count("]"))
            if content.count("{") > content.count("}"):
                fixed_content = fixed_content + "}" * (content.count("{") - content.count("}"))
            
            try:
                valorizzazioni = json.loads(fixed_content)
                print("JSON riparato con successo")
            except json.JSONDecodeError as e:
                print(f"Impossibile riparare il JSON: {e}")
                valorizzazioni = []
    
    return valorizzazioni

In [10]:
def carica_entita_su_neo4j(categoria, entities, driver):
    """
    Carica le entità filtrate come nodi nel database Neo4j.
    
    Args:
        categoria (str): La categoria/label da utilizzare per i nodi.
        entities (list): Lista di entità da caricare.
        driver: Connessione al database Neo4j.
    """
    
    with driver.session() as session:
        if not entities:  # Controlla se la lista di entità è vuota
            print(f"Nessuna entità trovata per la categoria {categoria}")
            return
            
        for entity in entities:
            # Controlla se l'entità ha la proprietà 'NOME'
            if 'NOME' not in entity or not entity['NOME']:
                print(f"Entità senza nome trovata in categoria {categoria}, skip")
                continue
            
            # Assicurati che il nome sia decodificato per gestire caratteri speciali
            nome_decodificato = html.unescape(entity['NOME'])
            
            # Prepara le proprietà base
            properties = {'nome': nome_decodificato}
            
            # Aggiungi i parametri se presenti
            if 'PARAMETRI' in entity and entity['PARAMETRI']:
                # Se PARAMETRI è una lista di stringhe
                if isinstance(entity['PARAMETRI'], list):
                    # Convertire la lista in una stringa con separatore
                    properties['parametri'] = ', '.join(entity['PARAMETRI'])
                    # Opzionalmente, aggiungi anche i singoli elementi numerati
                    for i, param in enumerate(entity['PARAMETRI']):
                        properties[f'parametro_{i+1}'] = param
                # Se PARAMETRI è un dizionario
                elif isinstance(entity['PARAMETRI'], dict):
                    for key, value in entity['PARAMETRI'].items():
                        properties[key.lower()] = value
                # Se PARAMETRI è una stringa singola
                elif isinstance(entity['PARAMETRI'], str):
                    properties['parametri'] = entity['PARAMETRI']
            
            # Crea un nodo per ciascuna entità con tutte le proprietà
            query = f"""
            MERGE (e:{categoria} {{nome: $nome}})
            SET e += $properties
            """
            
            try:
                # Esegui la query
                session.run(query, nome=nome_decodificato, properties=properties)
                print(f"Caricata entità {nome_decodificato} in categoria {categoria}")
            except Exception as e:
                print(f"Errore nel caricamento dell'entità {nome_decodificato} in categoria {categoria}: {e}")
        
        print(f"Entità della categoria {categoria} caricate correttamente in Neo4j.")

## MAIN

In [11]:
# Carica il file CSV
df_domande = pd.read_csv(domande)
testo_domande = df_domande.to_string()  # Converti il CSV in testo

# Carica il file CSV
df_pianeti = pd.read_csv(pianeti)
testo_pianeti = df_pianeti.to_string()  # Converti il CSV in testo

# Esegui l'analisi del PDF
testo_manuale = estrai_testo_pdf(manuale_cucina)
# Esegui l'analisi del PDF
testo_codice = estrai_testo_pdf(codice_galattico)
# Esempio di utilizzo:
testo_blog1 = estrai_testo_html(blog_1)
# Esempio di utilizzo:
testo_blog2 = estrai_testo_html(blog_2)

In [12]:
with open('entita_filtrate.txt', 'r', encoding='utf-8') as file:
    entita_filtrate = [line.strip() for line in file if line.strip()]

print(entita_filtrate)

['Tecnica', 'Licenza', 'Ordine', 'Ingrediente', 'Ristorante']


In [15]:
#categorie = estrai_categorie_rilevanti(testo_manuale, entita_filtrate, llm)
#print(categorie)
for categoria in entita_filtrate:
    valorizzazioni = estrai_valorizzazioni(testo_manuale, categoria, llm, driver)
    carica_entita_su_neo4j(categoria, valorizzazioni, driver)

Caricata entità Marinatura a Infusione Gravitazionale in categoria Tecnica
Caricata entità Marinatura Temporale Sincronizzata in categoria Tecnica
Caricata entità Marinatura Psionica in categoria Tecnica
Caricata entità Marinatura tramite Reazioni d'Antimateria Diluite in categoria Tecnica
Caricata entità Marinatura Sotto Zero a Polarità Inversa in categoria Tecnica
Caricata entità Affumicatura a Stratificazione Quantica in categoria Tecnica
Caricata entità Affumicatura Temporale Risonante in categoria Tecnica
Caricata entità Affumicatura Psionica Sensoriale in categoria Tecnica
Caricata entità Affumicatura tramite Big Bang Microcosmico in categoria Tecnica
Caricata entità Affumicatura Polarizzata a Freddo Iperbarico in categoria Tecnica
Caricata entità Fermentazione Quantica a Strati Multiversali in categoria Tecnica
Caricata entità Fermentazione Temporale Sincronizzata in categoria Tecnica
Caricata entità Fermentazione Psionica Energetica in categoria Tecnica
Caricata entità Fermenta

In [16]:
#categorie = estrai_categorie_rilevanti(testo_codice, entita_filtrate, llm)
#print(categorie)
for categoria in entita_filtrate:
    valorizzazioni = estrai_valorizzazioni(testo_codice, categoria, llm, driver)
    carica_entita_su_neo4j(categoria, valorizzazioni, driver)

Entità senza nome trovata in categoria Tecnica, skip
Entità della categoria Tecnica caricate correttamente in Neo4j.
Caricata entità licenza di classe Psionica (P) in categoria Licenza
Caricata entità licenza di classe Gravitazionale (G) in categoria Licenza
Caricata entità licenza di classe Antimateria (e+) in categoria Licenza
Caricata entità licenza di classe Magnetica (Mx) in categoria Licenza
Caricata entità licenza Gravitazionale (G) in categoria Licenza
Caricata entità licenza Temporale (t) in categoria Licenza
Caricata entità licenza Psionica (P) in categoria Licenza
Caricata entità licenza Antimateria (e+) in categoria Licenza
Caricata entità licenza Magnetica (Mx) in categoria Licenza
Caricata entità licenza Quantistica (Q) in categoria Licenza
Caricata entità licenza Quantistica (Q) in categoria Licenza
Caricata entità licenza Quantistica (Q) in categoria Licenza
Caricata entità licenza Temporale (t) in categoria Licenza
Caricata entità licenza Gravitazionale (G) in categori

In [17]:
#categorie = estrai_categorie_rilevanti(testo_blog1, entita_filtrate, llm)
#print(categorie)
for categoria in entita_filtrate:
    valorizzazioni = estrai_valorizzazioni(testo_blog1, categoria, llm, driver)
    carica_entita_su_neo4j(categoria, valorizzazioni, driver)

Entità senza nome trovata in categoria Tecnica, skip
Entità della categoria Tecnica caricate correttamente in Neo4j.
Caricata entità certificazioni dello chef Pierino "Il Custode" Temporali in categoria Licenza
Caricata entità codice galattico in categoria Licenza
Entità della categoria Licenza caricate correttamente in Neo4j.
Entità senza nome trovata in categoria Ordine, skip
Entità della categoria Ordine caricate correttamente in Neo4j.
Caricata entità Carne di Drago in categoria Ingrediente
Caricata entità Petali di Eco in categoria Ingrediente
Caricata entità Muffa Lunare in categoria Ingrediente
Entità della categoria Ingrediente caricate correttamente in Neo4j.
Caricata entità L'Etere del Gusto in categoria Ristorante
Entità della categoria Ristorante caricate correttamente in Neo4j.


In [18]:
#categorie = estrai_categorie_rilevanti(testo_blog2, entita_filtrate, llm)
#print(categorie)
for categoria in entita_filtrate:
    valorizzazioni = estrai_valorizzazioni(testo_blog2, categoria, llm, driver)
    carica_entita_su_neo4j(categoria, valorizzazioni, driver)

Caricata entità Cottura Sottovuoto Frugale Energeticamente Negativa in categoria Tecnica
Entità della categoria Tecnica caricate correttamente in Neo4j.
Entità senza nome trovata in categoria Licenza, skip
Entità della categoria Licenza caricate correttamente in Neo4j.
Caricata entità Sale Temporale in categoria Ordine
Caricata entità Nettare di Sirena in categoria Ordine
Caricata entità Polvere di Stelle in categoria Ordine
Caricata entità Carne di Drago in categoria Ordine
Caricata entità Cristalli di Memoria in categoria Ordine
Caricata entità Petali di Eco in categoria Ordine
Caricata entità Essenza di Vuoto in categoria Ordine
Caricata entità Nettare di Sirena in categoria Ordine
Caricata entità Funghi dell'Etere in categoria Ordine
Caricata entità Carne di Drago in categoria Ordine
Entità della categoria Ordine caricate correttamente in Neo4j.
Caricata entità Sale Temporale in categoria Ingrediente
Caricata entità Nettare di Sirena in categoria Ingrediente
Caricata entità Polvere

In [19]:
import os
from pathlib import Path

# Percorso della cartella contenente i file PDF dei menu
menu_folder = "../Hackapizza Dataset/Menu/"

# Ottieni la lista di tutti i file PDF nella cartella
pdf_files = [f for f in os.listdir(menu_folder) if f.endswith('.pdf')]

# Per ogni file PDF nella cartella
for pdf_file in pdf_files:
    try:
        # Costruisci il percorso completo del file
        pdf_path = os.path.join(menu_folder, pdf_file)
        
        print(f"Elaborazione del file: {pdf_file}")
        
        # Estrai il testo dal PDF
        testo_menu = estrai_testo_pdf(pdf_path)
        
        # Processa ogni categoria per questo menu
        for categoria in entita_filtrate:
            print(f"  Estrazione {categoria} da {pdf_file}")
            
            # Estrai le valorizzazioni
            valori = estrai_valorizzazioni(testo_menu, categoria, llm, driver)
            
            # Carica le entità su Neo4j
            if valori:  # Verifica che ci siano valori da caricare
                num_entita = len(valori) if isinstance(valori, list) else 1
                print(f"  Caricamento di {num_entita} entità di tipo {categoria}")
                carica_entita_su_neo4j(entities=valori, categoria=categoria, driver=driver)
            else:
                print(f"  Nessuna entità di tipo {categoria} trovata in {pdf_file}")
        
        print(f"Elaborazione di {pdf_file} completata con successo")
        print("-" * 50)
        
    except Exception as e:
        print(f"Errore durante l'elaborazione di {pdf_file}: {str(e)}")
        # Continua con il prossimo file anche in caso di errore
        continue

Elaborazione del file: Anima Cosmica.pdf
  Estrazione Tecnica da Anima Cosmica.pdf
  Caricamento di 19 entità di tipo Tecnica
Caricata entità Cottura a Vapore in categoria Tecnica
Caricata entità Cottura a Vapore in categoria Tecnica
Caricata entità Taglio a Risonanza Sonica in categoria Tecnica
Caricata entità Cottura in categoria Tecnica
Caricata entità Affumicatura in categoria Tecnica
Caricata entità Cottura a Forno in categoria Tecnica
Caricata entità Saltare in Padella in categoria Tecnica
Caricata entità Marinatura in categoria Tecnica
Caricata entità Congelamento in categoria Tecnica
Caricata entità Sinergia Elettro-Osmotica in categoria Tecnica
Caricata entità Bollitura Infrasonica in categoria Tecnica
Caricata entità Marinatura in categoria Tecnica
Caricata entità Grigliatura Eletro-Molecolare in categoria Tecnica
Caricata entità Cottura a Vapore in categoria Tecnica
Caricata entità Cottura Idrodinamica in categoria Tecnica
Caricata entità Bollitura Termografica in categoria 

KeyboardInterrupt: 